In [5]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import os

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
formula_test= pd.read_csv("/content/gdrive/MyDrive/critical-semiconductor-temperature/formula_test.csv")
print(formula_test)
formula_test.shape

        H  He   Li   Be    B  ...   Bi  Po  At  Rn                 material
0     0.0   0  0.0  0.0  0.0  ...  0.0   0   0   0      La1Ba2Cu2.94Fe0.06O
1     0.0   0  0.0  0.0  0.0  ...  0.0   0   0   0       Sr1Pd1.92Pt0.08Ge2
2     0.0   0  0.0  0.0  0.0  ...  0.0   0   0   0       Ba1Fe1.55Ru0.45As2
3     0.0   0  0.0  0.0  0.0  ...  0.0   0   0   0      Y1Ba2Cu2.97Fe0.03O7
4     0.0   0  0.0  0.0  0.0  ...  0.0   0   0   0        Pr1.85Ce0.15Cu1O4
...   ...  ..  ...  ...  ...  ...  ...  ..  ..  ..                      ...
4258  0.0   0  0.0  0.0  0.0  ...  0.0   0   0   0  Sn0.504Ge0.336In0.16Te1
4259  0.0   0  0.0  0.0  0.0  ...  1.8   0   0   0    Bi1.8Pb0.2Sr2Ca1Cu2O8
4260  0.0   0  0.0  0.0  1.9  ...  0.0   0   0   0              Mg1B1.9C0.1
4261  0.0   0  0.0  0.0  0.0  ...  0.0   0   0   0  Ca9.07La0.93Pt3Fe10As18
4262  0.0   0  0.0  0.0  0.0  ...  0.0   0   0   0          Mo0.98Nb0.02Ga4

[4263 rows x 87 columns]


(4263, 87)

In [10]:
formula_train= pd.read_csv("/content/gdrive/MyDrive/critical-semiconductor-temperature/formula_train.csv")
print(formula_train)
formula_train.shape

         H  He   Li   Be  ...  At  Rn  critical_temp                    material
0      0.0   0  0.0  0.0  ...   0   0         21.500            Eu1Fe1.5Ru0.5As2
1      0.0   0  0.0  0.0  ...   0   0         62.000              Bi2Sr2Ca1Cu2O8
2      0.0   0  0.0  0.0  ...   0   0          0.290                    N0.3S0.7
3      0.0   0  0.0  0.0  ...   0   0         19.500            Ba1Fe1.9Co0.1As2
4      0.0   0  0.0  0.0  ...   0   0          5.360                     Rh17S15
...    ...  ..  ...  ...  ...  ..  ..            ...                         ...
16995  0.0   0  0.0  0.0  ...   0   0         35.000                    La2Cu1O4
16996  0.0   0  0.8  0.0  ...   0   0         89.000        Bi2Sr2Ca1Cu1.2Li0.8O
16997  0.0   0  0.0  0.0  ...   0   0          0.584                  Os0.9Ru0.1
16998  0.0   0  0.0  0.0  ...   0   0         45.000              Sm1Ba2Cu3O6.73
16999  0.0   0  0.0  0.0  ...   0   0         13.500  Ba1Fe1.832Co0.15Mn0.018As2

[17000 rows x 88 columns]


(17000, 88)

In [26]:
test= pd.read_csv("/content/gdrive/MyDrive/critical-semiconductor-temperature/test.csv")
print(test)
test.shape

      feature1    feature2    feature3  ...  feature79  feature80  feature81
0            5   82.324574   88.533544  ...   0.814286   0.489898   0.358466
1            4  115.438500   90.562624  ...   1.504000   1.414214   0.853567
2            4   92.290900   83.842290  ...   1.600000   1.581139   1.361286
3            5   72.324650   51.227894  ...   1.070000   0.489898   0.270099
4            4   90.142263   58.462879  ...   1.057143   1.000000   0.903508
...        ...         ...         ...  ...        ...        ...        ...
4258         4  108.439500  115.102200  ...   2.760000   1.089725   1.110675
4259         6  103.903967   59.200661  ...   1.013333   1.213352   0.991878
4260         3   15.708900   15.348990  ...   1.766667   0.816497   0.525991
4261         5  100.966814   72.804399  ...   2.127073   1.469694   1.366704
4262         3   86.196460   74.958186  ...   2.380000   1.247219   1.193643

[4263 rows x 81 columns]


(4263, 81)

In [27]:
train= pd.read_csv("/content/gdrive/MyDrive/critical-semiconductor-temperature/train.csv")
print(train)
train.shape

       feature1    feature2  ...  feature81  critical_temperature
0             4   95.950150  ...   1.135782                21.500
1             5   83.244760  ...   1.019804                62.000
2             2   23.035850  ...   1.374773                 0.290
3             4   81.756699  ...   1.198165                19.500
4             2   67.485250  ...   0.000000                 5.360
...         ...         ...  ...        ...                   ...
16995         3   72.816957  ...   0.451754                35.000
16996         6   70.527467  ...   1.388344                89.000
16997         2  145.650000  ...   0.000000                 0.584
16998         4   91.808100  ...   0.269042                45.000
16999         5   76.392968  ...   1.196725                13.500

[17000 rows x 82 columns]


(17000, 82)

In [28]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [29]:
print(train.info())
print(test.info())

print(train.dtypes)
print(test.dtypes)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 82 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   feature1              17000 non-null  int64  
 1   feature2              17000 non-null  float64
 2   feature3              17000 non-null  float64
 3   feature4              17000 non-null  float64
 4   feature5              17000 non-null  float64
 5   feature6              17000 non-null  float64
 6   feature7              17000 non-null  float64
 7   feature8              17000 non-null  float64
 8   feature9              17000 non-null  float64
 9   feature10             17000 non-null  float64
 10  feature11             17000 non-null  float64
 11  feature12             17000 non-null  float64
 12  feature13             17000 non-null  float64
 13  feature14             17000 non-null  float64
 14  feature15             17000 non-null  float64
 15  feature16          

In [30]:
print("________________________________________")
print(train.isna().sum())
print("________________________________________")
print(test.isna().sum())

________________________________________
feature1                0
feature2                0
feature3                0
feature4                0
feature5                0
                       ..
feature78               0
feature79               0
feature80               0
feature81               0
critical_temperature    0
Length: 82, dtype: int64
________________________________________
feature1     0
feature2     0
feature3     0
feature4     0
feature5     0
            ..
feature77    0
feature78    0
feature79    0
feature80    0
feature81    0
Length: 81, dtype: int64


In [31]:
print(formula_train.info())
print(formula_test.info())

print(formula_train.dtypes)
print(formula_test.dtypes)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 88 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   H              17000 non-null  float64
 1   He             17000 non-null  int64  
 2   Li             17000 non-null  float64
 3   Be             17000 non-null  float64
 4   B              17000 non-null  float64
 5   C              17000 non-null  float64
 6   N              17000 non-null  float64
 7   O              17000 non-null  float64
 8   F              17000 non-null  float64
 9   Ne             17000 non-null  int64  
 10  Na             17000 non-null  float64
 11  Mg             17000 non-null  float64
 12  Al             17000 non-null  float64
 13  Si             17000 non-null  float64
 14  P              17000 non-null  float64
 15  S              17000 non-null  float64
 16  Cl             17000 non-null  float64
 17  Ar             17000 non-null  int64  
 18  K     

In [32]:
print("________________________________________")
print(formula_train.isna().sum())
print("________________________________________")
print(formula_test.isna().sum())

________________________________________
H                0
He               0
Li               0
Be               0
B                0
                ..
Po               0
At               0
Rn               0
critical_temp    0
material         0
Length: 88, dtype: int64
________________________________________
H           0
He          0
Li          0
Be          0
B           0
           ..
Bi          0
Po          0
At          0
Rn          0
material    0
Length: 87, dtype: int64


In [37]:
from sklearn import preprocessing
y = train['critical_temperature'].astype(float)
X = train.drop('critical_temperature', axis=1)
X = train.astype(float)
X_scale = preprocessing.scale(X)
X

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,...,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50,feature51,feature52,feature53,feature54,feature55,feature56,feature57,feature58,feature59,feature60,feature61,feature62,feature63,feature64,feature65,feature66,feature67,feature68,feature69,feature70,feature71,feature72,feature73,feature74,feature75,feature76,feature77,feature78,feature79,feature80,feature81,critical_temperature
0,4.0,95.950150,87.221940,89.534122,81.423258,1.317590,1.301059,96.119000,20.285800,36.105357,34.833271,740.625,786.620000,726.788285,771.273911,1.367745,1.202333,399.3,307.450000,142.347627,149.963748,169.750000,156.400000,164.442987,150.821660,1.355290,1.327938,117.0,29.000000,42.180416,43.345588,7803.750000,6938.800000,7354.256010,6686.502371,1.325027,1.325720,7126.000,1313.400000,2816.060402,...,57.540000,51.870923,47.246119,1.238413,1.159644,85.60,26.640000,31.960640,29.887027,19.125000,19.650000,17.385762,17.932694,1.299408,1.140380,18.400,9.220000,7.772508,7.827675,65.750000,58.500000,50.754738,48.531069,1.191039,1.203416,106.00000,20.900000,38.873995,30.483602,4.250000,4.100000,4.053600,3.944244,1.339718,1.245504,3.0,1.400000,1.299038,1.135782,21.500
1,5.0,83.244760,59.224400,59.506081,36.126175,1.311510,1.372050,192.981000,25.192187,67.233835,64.196304,779.880,1005.866667,740.399333,945.371396,1.553219,1.019202,764.1,661.026667,276.063148,333.238432,149.800000,106.133333,133.425311,86.457339,1.519537,1.573185,171.0,16.266667,58.608532,66.319295,4584.285800,2953.428800,874.164786,64.622881,1.161774,1.099512,9778.571,1303.237867,4004.273231,...,105.475333,33.148783,68.547287,1.200886,0.827671,138.63,75.742000,57.645673,52.670782,8.152400,4.954400,4.646433,1.321875,1.392937,1.395814,12.878,1.628267,4.361026,5.253410,128.605316,72.414176,14.287643,0.969156,0.871976,0.777323,399.97342,53.319157,154.008028,137.579265,2.600000,2.400000,2.402249,2.259897,1.519383,1.365077,3.0,0.933333,1.200000,1.019804,62.000
2,2.0,23.035850,26.647510,21.192566,25.010514,0.614232,0.435817,18.058300,18.243490,9.029150,8.275353,1200.250,1119.750000,1183.257580,1105.792837,0.679023,0.661811,402.5,278.850000,201.250000,184.448672,71.500000,77.700000,69.799713,76.228941,0.669462,0.522077,31.0,44.100000,15.500000,14.205985,980.625500,1372.375300,49.517270,215.653466,0.005331,0.002517,1958.749,1371.624700,979.374500,...,143.600000,41.385384,77.950376,0.169455,0.089179,193.00,138.500000,96.500000,88.443711,1.045000,1.319000,0.789177,1.080247,0.459434,0.283338,1.370,1.103000,0.685000,0.627813,0.115415,0.151249,0.072768,0.110120,0.350471,0.202131,0.17917,0.135751,0.089585,0.082106,4.500000,5.100000,4.242641,4.873514,0.636514,0.465999,3.0,3.300000,1.500000,1.374773,0.290
3,4.0,81.756699,79.833804,76.282833,75.275775,1.312596,1.149324,81.482000,28.789976,32.890369,29.969284,741.475,782.600000,723.246140,763.358485,1.362746,1.059188,443.7,363.318000,157.803998,163.153443,168.750000,158.520000,161.714287,151.499174,1.342444,1.166582,139.0,56.240000,51.329207,50.807181,6502.750000,6162.920000,6126.394078,5912.543550,1.332169,1.068492,5390.000,2814.120000,2072.768363,...,42.730000,34.258466,31.894684,1.166445,0.803499,64.05,30.496000,28.471814,30.780312,16.425000,18.248000,14.919043,16.404170,1.293519,0.946275,19.700,10.756000,7.159740,8.021377,61.750000,55.620000,51.637761,49.174596,1.244889,0.997395,82.00000,28.020000,30.873735,23.297974,3.500000,3.620000,3.309751,3.413039,1.333736,1.019322,3.0,1.920000,1.118034,1.198165,19.500
4,2.0,67.485250,69.699016,57.442709,59.574464,0.548263,0.521345,70.840500,39.638078,35.420250,35.351002,859.250,850.515625,847.809236,839.1

In [43]:
from sklearn.linear_model import LinearRegression
X = train.drop(['critical_temperature'], axis=1)
y = train['critical_temperature']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

model = LinearRegression()
model.fit(X_train, y_train)
answers_pred = model.predict(X_train)

print(model.score(X_test, y_test))

0.739435606954626


In [59]:
predictions = model.predict(test)
predictions



array([ 69.32463612,  21.97884151,  21.05341563, ...,  22.03295419,
        14.17484581, -17.11382417])

In [60]:
otvet1 = pd.DataFrame({'Index': [i for i in range(len(predictions))],'critical_temp': predictions})
from google.colab import files
otvet1.to_csv('Solovyeva.csv', index=False)
files.download("Solovyeva.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(X_train)

model_predictions = kmeans.predict(X_test)
print(model.score(X_test, y_test))

0.739435606954626


In [61]:
predictions_2 = kmeans.predict(test)
predictions_2

array([0, 1, 2, ..., 2, 1, 2], dtype=int32)

In [62]:
otvet2 = pd.DataFrame({'Index': [i for i in range(len(predictions))],'critical_temp': predictions_2})
otvet2.to_csv('Solovyeva.csv', index=False)
files.download("Solovyeva.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#выяснилось-кмеанс ужасен не смотрите на него